In [0]:
import requests
import re 

def ingest_stock_names():
    url = 'https://www.biznesradar.pl/gielda/akcje_gpw'
    page_content = requests.get(url).text
    matches = re.findall('<a href="\/notowania.*<\/a>', page_content)
    for match in matches:
        stock_name = match.split('title="')[1].split('">')[0]
        ticker = match.split(' (')[0].split('">')[1].replace('</a>', '')
        yield stock_name, ticker

for a,b in ingest_stock_names():
    print(a,b)

In [0]:
stocks = [] 

for stock_name, ticker in ingest_stock_names():
    stocks.append((stock_name, ticker))

schema = 'stock_name string, ticker string'
df = spark.createDataFrame(stocks, schema)
spark.sql("DROP TABLE IF EXISTS plstocks.bronze_company_name_lookup")
df.write.mode("overwrite").saveAsTable("plstocks.bronze_company_name_lookup")
display(df)